In [325]:
import math
import random
import matplotlib.pyplot as plt
import numpy as np

In [326]:
def getCrossProduct(p0,p1,p2):
    # Returns which is more counterclockwise
    cross_product = (p1[0] - p0[0]) * (p2[1]-p0[1]) - (p2[0]-p0[0]) * (p1[1]-p0[1])

    if cross_product == 0:
        return 0 #p1
    elif cross_product > 0:
        return 1 #p2
    else:
        return -1 #p1

In [327]:
def findFurthest(origin, points):
    def distance(point):
        return math.sqrt((origin[0] - point[0]) ** 2 + (origin[1] - point[1]) ** 2)
    
    return max(points, key=distance, default=None)

In [328]:
def jarvis(inputSet):
    outputSet = []
    if len(inputSet) < 3: # Since anything less than these number of points can't create a convex hull
        return inputSet

    # Find the leftmost point
    firstPoint = min(inputSet)
    curPoint = firstPoint
    nextPoint = None
    while nextPoint != firstPoint:
        outputSet.append(curPoint)

        nextPoint = inputSet[0] if inputSet[0] != curPoint else inputSet[1]

        for testPoint in inputSet:
            if testPoint == curPoint:
                continue
            crossProduct = getCrossProduct(curPoint, nextPoint, testPoint)
            if crossProduct > 0:
                nextPoint = testPoint
            elif crossProduct == 0:
                nextPoint = findFurthest(curPoint, [nextPoint, testPoint])
            

        curPoint = nextPoint
    return outputSet

In [329]:
def grahamscan(inputSet):
    outputSet = []
    pointAngles = {}
    firstPoint = min(inputSet, key=lambda p: (p[1], p[0])) # Get the lowest and leftmost point

    # Grouping the points by polar angle
    for point in inputSet:
        if point == firstPoint:
            continue
        
        angle_rad = math.atan2(point[1] - firstPoint[1], point[0] - firstPoint[0] )

        if angle_rad in pointAngles:
            pointAngles[angle_rad].append(point)
        else:
            pointAngles[angle_rad] = [point]

    tempSet = [subArr[1][0] if len(subArr) == 1 else findFurthest(firstPoint, subArr[1]) for subArr in sorted(list(pointAngles.items()))] + [firstPoint]
    
    for point in tempSet:
        while len(outputSet) >= 2 and getCrossProduct(outputSet[-2], outputSet[-1], point) <= 0:
            outputSet.pop()  # Remove the last point in the outputSet because it makes a non-left turn
        outputSet.append(point)

    return outputSet

In [330]:
def linearSearch(subHull, curPoint):
    hullSize = len(subHull)
    if hullSize == 1:
        return subHull[0]
    elif hullSize == 2:
        crossProduct = getCrossProduct(curPoint, subHull[0], subHull[1])
        if crossProduct > 0:
            return subHull[0]
        elif crossProduct < 0:
            return subHull[1]
        else:
            if curPoint == subHull[0]:
                return subHull[1]
            elif curPoint == subHull[1]:
                return subHull[0]
            return findFurthest(curPoint, [subHull[0], subHull[1]])

    pointer = 0
    nextPointer = (pointer + 1) % hullSize

    while getCrossProduct(curPoint, subHull[pointer], subHull[nextPointer]) <= 0:
        pointer = (pointer + 1) % hullSize
        nextPointer = (pointer + 1) % hullSize

    return subHull[pointer]


In [331]:
def linearChan(inputSet, groupSize=-1):
  outputSet = []
  hullSize = len(inputSet)

  if groupSize == -1:
    groupSize = math.ceil(math.sqrt(hullSize))

  subsets = [inputSet[i : i + groupSize] for i in range(0, hullSize, groupSize)]
  subHulls = [grahamscan(subset) for subset in subsets]

  firstPoint = min(inputSet, key=lambda p: (p[1],p[0]))
  curPoint = firstPoint
  nextPoint = None

  while nextPoint != firstPoint:
    outputSet.append(curPoint)
    rightPoints = [linearSearch(subHull, curPoint) for subHull in subHulls]
    nextPoint = rightPoints[0] if rightPoints[0] != curPoint else rightPoints[1]

    for rightPoint in rightPoints:
        if rightPoint == curPoint:
            continue
        crossProduct = getCrossProduct(curPoint, nextPoint, rightPoint)
        if crossProduct < 0:
            nextPoint = rightPoint
        elif crossProduct == 0:
            nextPoint = findFurthest(curPoint, [nextPoint, rightPoint])

    curPoint = nextPoint
  return outputSet


In [332]:
def binarySearch(subHull, curPoint):

    # print(subHull, curPoint)
    hullSize = len(subHull)

    if hullSize == 1:
        return subHull[0]
    elif hullSize == 2:
        crossProduct = getCrossProduct(curPoint, subHull[0], subHull[1])
        if crossProduct > 0:
            return subHull[0]
        elif crossProduct < 0:
            return subHull[1]
        else:
            if curPoint == subHull[0]:
                return subHull[1]
            elif curPoint == subHull[1]:
                return subHull[0]
            return findFurthest(curPoint, [subHull[0], subHull[1]])

    pointer = 0
    stepSize = max(hullSize // 2, 1)
    rightPointer = 0

    while True:
        # Need to think about what if the curPoint is the same as subHull[pointer]
        pointerChanged = False
        stepSize = max(stepSize // 2, 1)

        for operation in [+1, -1]:
            nextPointer = (pointer + operation) % hullSize

            cp = getCrossProduct(curPoint, subHull[pointer], subHull[nextPointer])
            if cp < 0:
                pointer = (pointer + (stepSize * operation)) % hullSize
                pointerChanged = True
                break
            elif cp == 0:
                if curPoint == subHull[nextPointer]:
                    oppositePointer = (nextPointer + (operation)) % hullSize
                    cp = getCrossProduct(curPoint, subHull[pointer], subHull[oppositePointer])
                    if cp < 0:
                        return subHull[oppositePointer]
                    else:
                        return subHull[pointer]
                else:
                    nextNextPointer = (nextPointer + operation) % hullSize
                    if (getCrossProduct(curPoint, subHull[nextPointer], subHull[nextNextPointer])> 0):
                        return findFurthest(curPoint, [subHull[pointer], subHull[nextPointer]])

        if not pointerChanged:
            return subHull[pointer]


In [333]:
def binaryChan(inputSet, groupSize=-1):
    hullSize = len(inputSet)

    if groupSize == -1:
        groupSize = math.ceil(math.sqrt(hullSize))

    subsets = [inputSet[i : i + groupSize] for i in range(0, hullSize, groupSize)]
    outputSet = []
    subHulls = [grahamscan(subset) for subset in subsets]

    firstPoint = min(inputSet, key=lambda p: (p[1], p[0]))
    curPoint = firstPoint
    nextPoint = None
    while nextPoint != firstPoint:
        outputSet.append(curPoint)
        rightPoints = [binarySearch(subHull, curPoint) for subHull in subHulls]
        nextPoint = rightPoints[0] if rightPoints[0] != curPoint else rightPoints[1]

        for rightPoint in rightPoints:
            if rightPoint == curPoint:
                continue
            crossProduct = getCrossProduct(curPoint, nextPoint, rightPoint)
            if crossProduct < 0:
                nextPoint = rightPoint
            elif crossProduct == 0:
                nextPoint = findFurthest(curPoint, [nextPoint, rightPoint])

        curPoint = nextPoint
    return outputSet


In [334]:
def oldBinarySearch(subHull, curPoint):

    hullSize = len(subHull)

    p1, p2 = 0, hullSize // 2
    stepSize = max(hullSize // 4, 1)
    p2MoreRight = True
    rightPoint = 0


    if hullSize == 1:
        return subHull[0]
    elif hullSize == 2:
        crossProduct = getCrossProduct(curPoint, subHull[0], subHull[1])
        if crossProduct > 0:
            return subHull[0]
        elif crossProduct < 0:
            return subHull[1]
        else:
            if curPoint == subHull[0]:
                return subHull[1]
            elif curPoint == subHull[1]:
                return subHull[0]
            return findFurthest(curPoint, [subHull[0], subHull[1]])

    while True:

        cp = getCrossProduct(curPoint, subHull[p1], subHull[p2])
        if cp < 0:
            rightPoint = p2
            p2MoreRight = True
        elif cp > 0:
            rightPoint = p1
            p2MoreRight = False
        else:  # This is if curPoint, p1 and p2 are collinear
            if abs(p2 - p1) == 1:
                return findFurthest(curPoint, [subHull[p1], subHull[p2]])

            bigMidPoint = (p1 + stepSize) % hullSize
            smallMidPoint = (p1 - stepSize) % hullSize
            p2MoreRight = True
            cp = getCrossProduct(curPoint, subHull[bigMidPoint], subHull[smallMidPoint])
            if cp > 0:  # smallMidPoint is more left
                rightPoint = bigMidPoint
            elif cp < 0:
                rightPoint = smallMidPoint
            else:
                print("Something wrong")
                break

        nextRight = (rightPoint + 1) % hullSize
        prevRight = (rightPoint - 1) % hullSize

        cp = getCrossProduct(curPoint, subHull[rightPoint], subHull[nextRight])
        if cp < 0:
            if p2MoreRight:
                p2 = nextRight
                p1 = (p2 + stepSize) % hullSize
            else:
                p1 = nextRight
                p2 = (p1 + stepSize) % hullSize
            stepSize = max(stepSize // 2, 1)
            continue
        elif cp == 0:
            return findFurthest(curPoint, [subHull[rightPoint], subHull[nextRight]])

        cp = getCrossProduct(curPoint, subHull[rightPoint], subHull[prevRight])
        if cp < 0:
            if p2MoreRight:
                p2 = prevRight
                p1 = (p2 - stepSize) % hullSize
            else:
                p1 = prevRight
                p2 = (p1 - stepSize) % hullSize
            stepSize = max(stepSize // 2, 1)
            continue
        elif cp == 0:
            return findFurthest(curPoint, [subHull[rightPoint], subHull[prevRight]])
        return subHull[rightPoint]
    return None


In [335]:
def oldBinaryChan(inputSet, groupSize=-1):

    hullSize = len(inputSet)

    if groupSize == -1:
        groupSize = math.ceil(math.sqrt(hullSize))
        # groupSize = math.ceil(hullSize/5)

    subsets = [inputSet[i : i + groupSize] for i in range(0, hullSize, groupSize)]
    outputSet = []
    subHulls = [grahamscan(subset) for subset in subsets]

    firstPoint = min(inputSet, key=lambda p: (p[1], p[0]))
    curPoint = firstPoint
    nextPoint = None
    while nextPoint != firstPoint:
        outputSet.append(curPoint)
        rightPoints = [oldBinarySearch(subHull, curPoint) for subHull in subHulls]
        nextPoint = rightPoints[0] if rightPoints[0] != curPoint else rightPoints[1]

        for rightPoint in rightPoints:
            if rightPoint == curPoint:
                continue
            crossProduct = getCrossProduct(curPoint, nextPoint, rightPoint)
            if crossProduct < 0:
                nextPoint = rightPoint
            elif crossProduct == 0:
                nextPoint = findFurthest(curPoint, [nextPoint, rightPoint])

        curPoint = nextPoint
    return outputSet


In [336]:
class TestDataGenerator():
    """
    A class to represent a synthetic data generator.

    ...

    Attributes
    ----------
    1. List of tuples - Store the (x,y) coordinate points of each point
    [to be defined as part of the coursework]
    2. Size - Number of points that this object should have

    Methods
    -------
    generatePoints(size)
    * size will be how many points are wanted to be created
    [to be defined as part of the coursework]

    """
       
    def __init__(self):
        self.points = []


    def getPoints(self):
        return self.points

    def generatePoints(self, size):
        self.points = []
        while len(self.points) < size:
            xp = random.randint(0,32767)
            yp = random.randint(0,32767)
            if (xp,yp) not in self.points:
                self.points.append((xp,yp))
        return self.points

    def genHorizontalLine(self,size):
        self.points = []
        while len(self.points) < size:
            xp = random.randint(0,32767)
            if (xp,0) not in self.points:
                self.points.append((xp,0))
        return self.points
    

    def generatePointsOnConvexHull(self, num_points, center=(16383, 16383), radius=16000):
         """ Generate points evenly spaced along the circumference of a circle.
           Parameters: num_points (int): Number of points to generate.
           center (tuple): Coordinates of the center of the circle.
            Default is (0, 0). 
            radius (float): Radius of the circle. 
            Default is 100. 
            Returns: list: List of (x, y) coordinates representing the points. """ 
         self.points = []
         for i in range(num_points):
             angle = 2 * math.pi * i / num_points 
             x = center[0] + radius * math.cos(angle) 
             y = center[1] + radius * math.sin(angle) 
             self.points.append((x, y))
    

In [337]:
import timeit
import matplotlib.pyplot as plt

class ExperimentalFramework():
    """
    A class to represent an experimental framework.

    ...

    Attributes
    ----------
    
    [to be defined as part of the coursework]

    Methods
    -------
    
    [to be defined as part of the coursework]

    """
        
    #ADD YOUR CODE HERE
    
    def __init__(self):
        self.testGen = TestDataGenerator()
        pass


    def linearPoints(self, size):
        pass
        jarvisResult = %timeit jarvis(test.genHorizontalLine(size))
        grahamResult = %timeit grahamscan(test.genHorizontalLine(size))
        chanResult = %timeit chan(test.genHorizontalLine(size))
    
    def stressTest(self, size, chanM):
        jarvisResult = %timeit -o jarvis(self.testGen.generatePoints(size))
        grahamResult = %timeit -o grahamscan(self.testGen.generatePoints(size))
        chanResult = %timeit -o chan(self.testGen.generatePoints(size),chanM)
        print(jarvisResult.average)
        print(grahamResult.average)
        print(chanResult.average)

    def chanComparison(self):
        sizes = [5,10,15]#100,200,400,800,1600,3200,6400,12800]
        randomTest = []
        allPointsOnHull = []
        pointsInLine = []
        hullPoints = self.testGen.generateHullPoints(size,sizes)


        for size in sizes:
            # average case
            randomTestVar = %timeit -o chan(self.testGen.generatePoints(size))
            randomTest.append(randomTestVar.average)

            #worst case
            allPointsOnHullVar = %timeit -o chan(self.testGen.genAllPointsOnHull(size))
            allPointsOnHull.append(allPointsOnHullVar.average)
            pointsInLineVar = %timeit -o chan(self.testGen.genHorizontalLine(size))
            pointsInLine.append(pointsInLineVar.average)
        self.plotResults(sizes, randomTest,allPointsOnHull,pointsInLine)


    def compare3AllHull(self):
        sizes = [5,10]
        jarvisResults = []
        grahamResults = []
        chanResults = []

        for size in sizes:
            hullPoints = self.testGen.generateHullPoints(1,size)
            jarvisResult = %timeit -o jarvis(hullPoints[0])
            grahamResult = %timeit -o grahamscan(hullPoints[0])
            chanResult = %timeit -o chan(hullPoints[0])
            jarvisResults.append(jarvisResults)
            grahamResults.append(grahamResult)
            chanResults.append(chanResult)
        self.plotResults(sizes, jarvisResults, grahamResults, chanResults)

    def compare3Random(self):
        # sizes = [10,100,200,500,1000,10000]
        sizes = [5000, 10000, 50000]
        chanM = [300, 400, 700]
        jarvisResults = []
        grahamResults = []
        chanResults = []

        for i, size in enumerate(sizes):
            print("Iteration:",i)
            hullPoints = self.testGen.generatePoints(size)
            jarvisResult = %timeit -o jarvis(hullPoints)
            grahamResult = %timeit -o grahamscan(hullPoints)
            chanResult = %timeit -o chan(hullPoints, chanM[i])
            jarvisResults.append(jarvisResult.average)
            grahamResults.append(grahamResult.average)
            chanResults.append(chanResult.average)
        self.plotResults(sizes, ["Jarvis", "Graham", "Chan"], jarvisResults, grahamResults, chanResults)

    def algoTest(self, repeats, sizes):
        wrongCount = 0
        wrongTests = []

        jarvisPoints = []
        grahamPoints = []
        chanPoints = []

        for size in sizes:
            print("Current Size:",size, "Wrong So Far:", wrongCount)
            print()
            for i in range(repeats):
                print(i)
                randomPoints = self.testGen.generatePoints(size)
                print(randomPoints)
                # jarvisPoints = jarvis(randomPoints)
                grahamPoints = grahamscan(randomPoints)
                chanPoints = oldBinaryChan(randomPoints)

                if not (sorted(grahamPoints) == sorted(chanPoints)):
                    print("Wrong Count:", 1)
                    wrongTests.append(randomPoints)
                    wrongCount += 1
        return wrongCount, wrongTests
    
    def compareChanLinearToBinary(self, inputSizes, groupSizes):
        linear_results = []
        binary_results = []
        graham_results = []
        


        for i, inputSize in enumerate(inputSizes):
            hullPoints = self.testGen.generatePoints(inputSize)
            hull = grahamscan(hullPoints)
            graham_result = %timeit -o -q grahamscan(hullPoints)
            graham_results.append(graham_result.average)
            groupSize = math.ceil(len(hull) / 5)
            # linear_result = %timeit -o linearChan(hullPoints, groupSize)
            binary_result = %timeit -o -q oldBinaryChaninaryChan(hullPoints, groupSizes[i])
            # linear_results.append(linear_result.average)
            binary_results.append(binary_result.average)

        self.plotResults(groupSizes[0], inputSizes, graham_results, binary_results)

    def findingBinaryChanOptimalGroupSize(self, inputSize, groupSizes):
        results = []

        # for i, inputSize in enumerate(inputSizes):

        # groupSizes = [i for i in range(700,2000,50)]
        
        # groupSizes = [10, 20, 500, 1000]

        for dataset in range(5):
            results.append([])
            testPoints = self.testGen.generatePoints(inputSize)
            
            for groupSize in groupSizes:
                result = %timeit -o -q binaryChan(testPoints,groupSize)
                # groupSizes.append(groupSize)
                results[dataset].append(result.average)
                

        self.plotResults(inputSize, groupSizes, results[0], results[1], results[2], results[3], results[4]) #

    def drawHull(self, points, lines):
        margin = 2000
        xPoints, yPoints = zip(*points)
        tempLines = lines + [lines[0]]
        xLine, yLine = zip(*tempLines)
        plt.scatter(xPoints, yPoints, color='red', s=3)
        plt.plot(xLine, yLine, '-', color='blue')

        plt.xlim(min(xPoints) - margin, max(xPoints) + margin)
        plt.ylim(min(yPoints) - margin, max(yPoints) + margin)
        plt.show()

    def plotPoints(self, points):
        margin = 10000
        xPoints, yPoints = zip(*points)
        plt.scatter(xPoints, yPoints, color='red', s=1)

        plt.xlim(min(xPoints) - margin, max(xPoints) + margin)
        plt.ylim(min(yPoints) - margin, max(yPoints) + margin)
        plt.show()

    def plotResults(self, title, xPoints, *yPoints):
        plt.title(title)
        colors = ['red','green','blue','black','purple','pink','orange','yellow']
        for i, list in enumerate(yPoints):
            y = [point for point in list]
            plt.plot(xPoints, y, 'o-', color=colors[i%len(colors)])

        plt.show()

In [338]:
test = ExperimentalFramework()
gen = TestDataGenerator()
points = gen.generatePoints(20)
# groupSizess = [i for i in range(10, 30, 10)]


# test.compare3Random()
# test.algoTest(100000, [i for i in range(100, 1000, 50)])
# test.findingBinaryChanOptimalGroupSize(500, groupSizess)
# test.stressTest(50000,500)
# test.chanComparison()


In [339]:

"""

inputSizes = [10000,20000,30000,40000,50000]

for x in range(10):
  groupSizes = [1000 + x * 100] * 5
  test.compareChanLinearToBinary(inputSizes, groupSizes)
"""
# inputSizes = [50,100,250,500,750,1000,5000,7500,10000,15000, 20000,25000, 30000,40000,50000]
# groupSizes = [5,10,30, 50, 75, 100, 500, 600, 1000, 1200, 1250, 1300, 1300, 1300, 1500]
# test.compareChanLinearToBinary(inputSizes, groupSizes)


'\n\ninputSizes = [10000,20000,30000,40000,50000]\n\nfor x in range(10):\n  groupSizes = [1000 + x * 100] * 5\n  test.compareChanLinearToBinary(inputSizes, groupSizes)\n'

In [340]:
test.algoTest(1000000,[i for i in range(11, 20, 1)])

Current Size: 11 Wrong So Far: 0

0
[(23773, 16255), (3933, 24145), (15646, 17054), (12966, 24167), (27557, 7449), (17231, 23360), (11285, 22710), (13761, 11411), (28101, 12350), (27270, 9587), (14611, 3854)]
1
[(8929, 10171), (24549, 1973), (30905, 26083), (15110, 30665), (27634, 21710), (10441, 15081), (28536, 24477), (9289, 14923), (23843, 6660), (11393, 5090), (28469, 26349)]
2
[(26739, 14926), (8443, 24137), (11478, 8054), (23498, 9159), (4988, 28357), (9724, 4915), (1044, 25282), (25413, 11951), (30059, 9241), (17049, 22333), (17409, 17797)]
3
[(16760, 21760), (17240, 25701), (15945, 17817), (8832, 12224), (30462, 16853), (10062, 13596), (19828, 2161), (24199, 25849), (30525, 26659), (8534, 3709), (6428, 6063)]
Wrong Count: 1
4
[(20139, 8136), (11670, 14688), (12343, 30174), (14616, 30007), (17355, 24719), (227, 4657), (13109, 2368), (28600, 13507), (15917, 12599), (32369, 27206), (31141, 29691)]
5
[(29820, 19493), (23861, 26352), (15908, 29111), (433, 31754), (2859, 20858), (175

KeyboardInterrupt: 

In [ ]:
inputSize = 500
groupSizes = [i for i in range(50, 250, 5)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 1000
groupSizes = [i for i in range(50, 400, 10)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 40000
groupSizes = [i for i in range(100, 4000, 100)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 50000
groupSizes = [i for i in range(500, 15000, 500)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 100000
groupSizes = [i for i in range(5000, 40000, 2000)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 5000
groupSizes = [i for i in range(100, 1700, 50)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 10000
groupSizes = [i for i in range(100, 3000, 100)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 20000
groupSizes = [i for i in range(100, 3000, 100)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)

inputSize = 30000
groupSizes = [i for i in range(100, 3000, 100)]
test.findingBinaryChanOptimalGroupSize(inputSize,groupSizes)


KeyboardInterrupt: 